# Procedural programming in python

## Topics

 * From nothing to something:
   * Pairwise correlation between rows in a pandas dataframe
   * Sketch of the process
   * In class exercise:
     * Write the code!
   * Rejoining, sharing ideas, problems, thoughts

<hr>


### Notes left over from last class.  Maybe be useful.  May not.

Notes from last class:
* The `os` package has tools for checking if a file exists: ``os.path.exists``
```
import os
filename = 'HCEPDB_moldata.zip'
if os.path.exists(filename):
    print("wahoo!")
```
* Use the `requests` package to get the file given a url (got this from the requests docs)
```
import requests
url = 'http://faculty.washington.edu/dacb/HCEPDB_moldata.zip'
req = requests.get(url)
assert req.status_code == 200 # if the download failed, this line will generate an error
with open(filename, 'wb') as f:
    f.write(req.content)
```
* Use the `zipfile` package to decompress the file while reading it into `pandas`
```
import pandas as pd
import zipfile
csv_filename = 'HCEPDB_moldata.csv'
zf = zipfile.ZipFile(filename)
data = pd.read_csv(zf.open(csv_filename))
```

Here was my solution
```
import os
import requests
import pandas as pd
import zipfile

filename = 'HCEPDB_moldata.zip'
url = 'http://faculty.washington.edu/dacb/HCEPDB_moldata.zip'
csv_filename = 'HCEPDB_moldata.csv'

if os.path.exists(filename):
    pass
else:
    req = requests.get(url)
    assert req.status_code == 200 # if the download failed, this line will generate an error
    with open(filename, 'wb') as f:
        f.write(req.content)

zf = zipfile.ZipFile(filename)
data = pd.read_csv(zf.open(csv_filename))
```



In [ ]:
def download_if_not_exists(filename):
    if os.path.exists(filename):
        pass
    else:
        req = requests.get(url)
        assert req.status_code == 200 # if the download failed, this line will generate an error
        with open(filename, 'wb') as f:
            f.write(req.content)

<HR>
## From something to nothing

### Task: Compute the pairwise Pearson correlation between rows in a dataframe.

Let's say we have three molecules (A, B, C) with three measurements each (v1, v2, v3).  So for each molecule we have a vector of measurements:

$$X=\begin{bmatrix}
         X_{v_{1}} \\
         X_{v_{2}} \\
         X_{v_{3}} \\
        \end{bmatrix} $$
        
Where X is a molecule and the components are the values for each of the measurements.  These make up the rows in our matrix.

Often, we want to compare molecules to determine how similar or different they are.  One measure is the Pearson correlation.

Pearson correlation: <img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/01d103c10e6d4f477953a9b48c69d19a954d978a"/>

Expressed graphically, when you plot the paired measurements for two samples (in this case molecules) against each other you can see positively correlated, no correlation, and negatively correlated.  Eg.
<img src="http://www.statisticshowto.com/wp-content/uploads/2012/10/pearson-2-small.png"/>


Simple input dataframe (_note_ when you are writing code it is always a good idea to have a simple test case where you can readily compute by hand or know the output):

| index | v1 | v2 | v3 |
|-------|----|----|----|
| A     | -1 | 0  | 1  |
| B     | 1  | 0  | -1 |
| C     | .5 | 0  | .5 |

### Take 3 minutes and answer the following questions in the notebook.

* If the above is a dataframe what shape and size is the output?

* Whare are some unique features of the output?

For our test case, what will the output be?

### Let's sketch the idea...

## In class exercise
### 20-30 minutes
#### Objectives: 
1. Write code using functions to compute the pairwise Pearson correlation between rows in a pandas dataframe.  You will have to use ``for`` and possibly ``if``.
2. Use a cell to test each function with an input that yields an expected output.  Think about the shape and values of the outputs.
3. Put the code in a ``.py`` file in the directory with the Jupyter notebook, import and run!


#### To help you get started...
To create the sample dataframe:
```
df = pd.DataFrame([[-1, 0, 1], [1, 0, -1], [.5, 0, .5]])
```

To loop over rows in a dataframe, check out (Google is your friend):
```
DataFrame.iterrows
```

For a row, to compute correlation to another list, series, vector, use:
```
my_row.corr(other_row)
```

You may want to use a ``numpy`` matrix, e.g.
```
import numpy as np
pair_corr_mat = np.zeros((2,2))
pair_corr_mat[1,1] = 42
pair_corr_mat
```

<hr>
## How do we know it is working?


#### Use the test case!
Our three row example is a useful tool for checking that our code is working.  We can write some tests that compare the output of our functions to our expectations.

E.g. The diagonals should be 1, and corr(A, B) = -1, ...

#### But first, let's talk ``assert`` and ``raise``

We've already briefly been exposed to assert in this code:
```
if os.path.exists(filename):
    pass
else:
    req = requests.get(url)
    # if the download failed, next line will raise an error
    assert req.status_code == 200
    with open(filename, 'wb') as f:
        f.write(req.content)
```

What is the assert doing there?

Let's play with ``assert``.  What should the following asserts do?
```
assert True == False, "You assert wrongly, sir!"
assert 'Dave' in instructors
assert function_that_returns_True_or_False(parameters)
```

So when an assert statement is true, the code keeps executing and when it is false, it ``raises`` an exception (also known as an error).

We've all probably seen lots of exception.  E.g.

```
def some_function(parameter):
    return

some_function()
```

```
some_dict = { }
print(some_dict['invalid key'])
```

```
'fourty' + 2
```

Like C++ and other languages, Python let's you ``raise`` your own exception.  You can do it with ``raise`` (surprise!).  Exceptions are special objects and you can create your own type of exceptions.  For now, we are going to look at the simplest ``Exception``.

We create an ``Exception`` object by calling the generator:
```
Exception()
```

This isn't very helpful.  We really want to supply a description.  The Exception object takes any number of strings.  One good form if you are using the generic exception object is:
```
Exception('Short description', 'Long description')
```



Creating an exception object isn't useful alone, however.  We need to send it down the software stack to the Python interpreter so that it can handle the exception condition.  We do this with ``raise``.

```
raise Exception("An error has occurred.")
```

Now you can create your own error messages like a pro!

#### DETOUR!

There are lots of types of exceptions beyond the generic class ``Exception``.  You can use them in your own code if they make sense.  E.g.  
```
import math
my_variable = math.inf
if my_variable == math.inf:
    raise ValueError('my_variable cannot be infinity')
```

<p>List of Standard Exceptions &minus;</p>
<table class="table table-bordered">
<tr>
<th><b>EXCEPTION NAME</b></th>
<th><b>DESCRIPTION</b></th>
</tr>
<tr>
<td>Exception</td>
<td>Base class for all exceptions</td>
</tr>
<tr>
<td>StopIteration</td>
<td>Raised when the next() method of an iterator does not point to any object.</td>
</tr>
<tr>
<td>SystemExit</td>
<td>Raised by the sys.exit() function.</td>
</tr>
<tr>
<td>StandardError</td>
<td>Base class for all built-in exceptions except StopIteration and SystemExit.</td>
</tr>
<tr>
<td>ArithmeticError</td>
<td>Base class for all errors that occur for numeric calculation.</td>
</tr>
<tr>
<td>OverflowError</td>
<td>Raised when a calculation exceeds maximum limit for a numeric type.</td>
</tr>
<tr>
<td>FloatingPointError</td>
<td>Raised when a floating point calculation fails.</td>
</tr>
<tr>
<td>ZeroDivisonError</td>
<td>Raised when division or modulo by zero takes place for all numeric types.</td>
</tr>
<tr>
<td>AssertionError</td>
<td>Raised in case of failure of the Assert statement.</td>
</tr>
<tr>
<td>AttributeError</td>
<td>Raised in case of failure of attribute reference or assignment.</td>
</tr>
<tr>
<td>EOFError</td>
<td>Raised when there is no input from either the raw_input() or input() function and the end of file is reached.</td>
</tr>
<tr>
<td>ImportError</td>
<td>Raised when an import statement fails.</td>
</tr>
<tr>
<td>KeyboardInterrupt</td>
<td>Raised when the user interrupts program execution, usually by pressing Ctrl+c.</td>
</tr>
<tr>
<td>LookupError</td>
<td>Base class for all lookup errors.</td>
</tr>
<tr>
<td><p>IndexError</p><p>KeyError</p></td>
<td><p>Raised when an index is not found in a sequence.</p><p>Raised when the specified key is not found in the dictionary.</p></td>
</tr>
<tr>
<td>NameError</td>
<td>Raised when an identifier is not found in the local or global namespace.</td>
</tr>
<tr>
<td><p>UnboundLocalError</p><p>EnvironmentError</p></td>
<td><p>Raised when trying to access a local variable in a function or method but no value has been assigned to it.</p><p>Base class for all exceptions that occur outside the Python environment.</p></td>
</tr>
<tr>
<td><p>IOError</p><p>IOError</p></td>
<td><p>Raised when an input/ output operation fails, such as the print statement or the open() function when trying to open a file that does not exist.</p><p>Raised for operating system-related errors.</p></td>
</tr>
<tr>
<td><p>SyntaxError</p><p>IndentationError</p></td>
<td><p>Raised when there is an error in Python syntax.</p><p>Raised when indentation is not specified properly.</p></td>
</tr>
<tr>
<td>SystemError</td>
<td>Raised when the interpreter finds an internal problem, but when this error is encountered the Python interpreter does not exit.</td>
</tr>
<tr>
<td>SystemExit</td>
<td>Raised when Python interpreter is quit by using the sys.exit() function. If not handled in the code, causes the interpreter to exit.</td>
</tr>
<tr>
<td>Raised when Python interpreter is quit by using the sys.exit() function. If not handled in the code, causes the interpreter to exit.</td>
<td>Raised when an operation or function is attempted that is invalid for the specified data type.</td>
</tr>
<tr>
<td>ValueError</td>
<td>Raised when the built-in function for a data type has the valid type of arguments, but the arguments have invalid values specified.</td>
</tr>
<tr>
<td>RuntimeError</td>
<td>Raised when a generated error does not fall into any category.</td>
</tr>
<tr>
<td>NotImplementedError</td>
<td>Raised when an abstract method that needs to be implemented in an inherited class is not actually implemented.</td>
</tr>
</table>

#### Put it all together... ``assert`` and ``raise``

Breaking assert down, it is really just an if test followed by a raise.  So the code below:
```
assert <some_test>, <message>
```
is equivalent to a short hand for:
```
if not <some_test>:
        raise AssertionError(<message>)       
```

Prove it?  OK.

```
instructors = ['Dorkus the Clown', 'Jim']
assert 'Dave' in instructors, "Dave isn't in the instructor list!"
```

```
instructors = ['Dorkus the Clown', 'Jim']
assert 'Dave' in instructors, "Dave isn't in the instructor list!"
if not 'Dave' in instructors:
    raise AssertionError("Dave isn't in the instructor list!")
```

#### Questions?



### All of this was in preparation for some testing...

Can we write some quick tests that make sure our code is doing what we think it is?  Something of the form:

```
corr_matrix = pairwise_row_correlations(my_sample_dataframe)
assert corr_matrix looks like what we expect, "The function is broken!"
```

What are the smallest units of code that we can test?

What asserts can we make for these pieces of code?

#### Remember, in computers, 1.0 does not necessarily = 1

Put the following in an empty cell:
```
.99999999999999999999
```

How can we test for two floating point numbers being (almost) equal? Pro tip:  [Google!](http://lmgtfy.com/?q=python+assert+almost+equal)



## From nothing to something wrap up

Here we created some functions from just a short description of our needs.  
* Before we wrote any code, we walked through the flow control and decided on the parts that were necessary.
* Before we wrote any code, we created a simple test example with simple predictable output.
* We wrote some code according to our specifications.
* We wrote tests using ``assert`` to verify our code against the simple test example.


### QUESTIONS?